<font style="font-size:96px; font-weight:bolder; color:#0040a0"><img src="http://montage.ipac.caltech.edu/docs/M51_logo.png" alt="M" style="float: left; padding: 25px 30px 25px 0px;" /></font>

<i><b>Montage</b> Montage is an astronomical image toolkit with components for reprojection, background matching, coaddition and visualization of FITS files. It can be used as a set of command-line tools (Linux, OS X and Windows), C library calls (Linux and OS X) and as Python binary extension modules.

The Montage source is written in ANSI-C and code can be downloaded from GitHub ( https://github.com/Caltech-IPAC/Montage ). The Python package can be installed from PyPI ("</i>pip install MontagePy<i>"). The package has no external dependencies. See http://montage.ipac.caltech.edu/ for details on the design and applications of Montage.


# MontagePy.main modules: mBgModel

The Montage modules are generally used as steps in a workflow to create a mosaic of a set of input images.  These steps are: determine the geometry of the mosaic on the sky, reproject the images to a common frame and spatial sampling; rectify the backgrounds to a common level, and coadd the images into a mosaic.  This page illustrates the use of one Montage module, mBgModel, which is used to model the set of background corrections needed for a set of images.

Visit <a href="Mosaic.ipynb">Building a Mosaic with Montage</a> to see how mBgModel is used as part of a workflow to creage a mosaic (or the <a href="Mosaic_oneshot.ipynb"> one shot </a> version if you just want to see the commands).  See the complete list of Montage Notebooks <a href="http://montage.ipac.caltech.edu/MontageNotebooks">here</a>.


In [1]:
from MontagePy.main import mBgModel, mViewer

help(mBgModel)

Help on built-in function mBgModel in module MontagePy.main:

mBgModel(...)
    mBgModel is a modelling/fitting program. It uses the image-to-image difference parameter table created by mFitExec to interactively determine a set of corrections to apply to each image in order to achieve a 'best' global fit.
    
    Parameters
    ----------
    input_file : str
        iReprojected image metadata list.
    fit_file : str
        Set of image overlap difference fits.
    corr_file : str
        Output table of corrections for images in input list.
    noslope : bool, optional
        Only fit levels, not slopes.
    usall : bool, optional
        Use all the input differences (by default we exclude very small overlap areas).
    niteration : int, optional
        Number of iterations to run.
    debug : int, optional
        Debugging output level.
    
    
    Returns
    -------
    status : int
        Return status (0: OK, 1:ERROR).
    msg : str
        Return message (for errors).

&nbsp;<p/>

## mBgModel Example

Once we have determined and fit pairwise the overlaps between images in a set, we use mBgModel to determine the corrections we should make to minimize the overall background differences.  The method used is a modified least-squares fitting for the set of correction coefficients using all the pixel-to-pixel differences in the overlap sets but with several approximations invoked to speed up the computational process.

We start with a set of pairwise differences, each fit with a plane and ignoring large excursions.  The idea here is that in an overlap region two images (both flux-calibrated) will differ only by the levels (and maybe a little slope) in their backgrounds.  Where the background varies more extremely, background matching is likely the be futile.  We have seen this happen, for instance, in 2MASS images where airglow due to blobs of gas in the Earth's atmosphere sometimes contaminate images on an size scale similar to the images.

Normally, though, these fit background differences look like this ('plus' and 'minus' refer to row IDs in the input image metadata table):

In [2]:
import pandas as pd
from astropy.io import ascii
ipactable = ascii.read('M17/fits.tbl').to_pandas()
ipactable

,plus,minus,a,b,c,crpix1,crpix2,xmin,xmax,ymin,ymax,xcenter,ycenter,npixel,rms,boxx,boxy,boxwidth,boxheight,boxang
0,0,8,1.268790e-03,4.644410e-04,0.410733,-1711.0,1756.5,1711,1789,-1756,-935,1750.15,-1343.07,52864,7.362550e-01,1749.9,-1346.0,821.2,77.3,-90.1
1,0,9,-2.523660e-03,1.653820e-03,6.969540,-1713.0,988.5,1713,1789,-988,-733,1751.01,-863.07,16232,7.125360e-01,1751.1,-861.0,255.3,76.9,-90.2
2,0,13,2.453210e-03,3.935060e-03,0.277418,-1286.0,986.5,1286,1361,-986,-732,1323.43,-862.04,16218,7.174900e-01,1323.5,-859.5,254.0,75.0,90.0
3,0,14,-2.490360e-03,8.991680e-04,4.436860,-1283.0,1755.5,1283,1361,-1755,-933,1322.70,-1342.62,52649,7.105830e-01,1322.3,-1344.5,822.2,77.3,89.9
4,0,21,-3.303240e-10,5.376120e-09,-1.130130,-1286.0,786.5,1286,1790,-786,-732,1537.86,-759.37,27003,7.587320e-06,1538.0,-759.5,54.0,504.0,90.0
5,0,24,-1.489270e-10,-5.884170e-09,0.561656,-1284.0,1756.5,1284,1787,-1756,-1702,1535.59,-1729.23,27090,7.108770e-06,1535.5,-1729.5,54.0,503.0,90.0
6,0,29,-1.230960e-03,5.115690e-04,-79.055300,-1710.0,994.5,1710,1790,-994,-733,1749.78,-865.65,16584,7.530540e-01,1749.9,-864.0,261.3,79.3,-90.2
7,0,31,2.329150e-03,1.100920e-03,-82.140500,-1708.0,1756.5,1708,1789,-1756,-941,1748.71,-1348.03,53097,7.707710e-01,1748.8,-1349.0,815.2,79.9,89.8
8,1,6,1.756510e-04,-2.708390e-04,-3.833850,426.0,789.5,-426,-351,-789,32,-387.36,-347.45,42063,9.129390e-01,-388.5,-378.5,822.0,75.0,90.0
9,1,11,-2.720230e-03,-5.054520e-04,-0.537130,-2.0,789.5,2,78,-789,31,40.09,-390.72,47921,8.727160e-01,40.0,-379.0,821.0,76.0,90.0


mBgModel uses this data to model the corrections for the individual IDs:

In [3]:
rtn = mBgModel('M17/pimages.tbl', 'M17/fits.tbl', 'work/M17/corrections.tbl')

print(rtn)

{'status': '0'}


In [4]:
ipactable = ascii.read('work/M17/corrections.tbl').to_pandas()
ipactable

,id,a,b,c
0,0,0.000907,0.001942,-8.773330
1,8,0.000224,0.001593,-10.007300
2,9,0.001333,-0.000788,-13.112400
3,13,0.000368,0.000673,-9.036830
4,14,0.000188,0.001220,-8.722280
5,21,0.000840,0.001092,-8.212320
6,24,0.000589,0.003644,-5.954610
7,29,0.000048,0.000451,73.046300
8,31,-0.000693,0.001017,72.364000
9,1,0.000141,0.000352,-6.415340


Montage functions return JSON structures.  They always include a status (0: success; 1: error) and a variable number of informational parameters.

## mBgModel Error Handling

If mBgModel encounters an error, the return structure will just have two elements: a status of 1 ("error") and a message string that tries to diagnose the reason for the error.

For instance, if the user asks for an invalid table file:


In [5]:
rtn = mBgModel('M17/unknown.tbl', 'M17/fits.tbl', 'work/M17/corrections.tbl')

print(rtn)

{'status': '1', 'msg': b'Invalid image metadata file: M17/unknown.tbl'}


&nbsp;

## Classic Montage: mBgExec as a Stand-Alone Program

### mBgModel Unix/Windows Command-line Arguments

<p>mBgModel can also be run as a command-line tool in Linux, OS X, and Windows:</p>

<p><tt>
<b>Usage:</b> mBgModel [-i niter] [-l(evel-only)] [-d level] [-a(ll-overlaps)] [-s statusfile] images.tbl fits.tbl corrections.tbl
</tt></p>
<p>&nbsp;</p>
<p>If you are writing in C/C++, mBgModel can be accessed as a library function:</p>


<pre>
/*-***********************************************************************/
/*                                                                       */
/*  mBModel                                                              */
/*                                                                       */
/*  Given a set of image overlap difference fits (parameters on the      */
/*  planes fit to pairwise difference images between adjacent images)    */
/*  interatively determine the 'best' background adjustment for each     */
/*  image (assuming each image is changed to best match its neighbors    */
/*  with them remaining unchanged) uses these adjustments to modify      */
/*  each set of difference parameters, and iterate until the changes     */
/*  modifications become small enough.                                   */
/*                                                                       */
/*   char  *imgfile        Reprojected image metadata list               */
/*   char  *fitfile        Set of image overlap difference fits          */
/*   char  *corrtbl        Output table of corrections for images        */
/*                         in input list                                 */
/*   int    noslope        Only fit levels, not slopes                   */
/*   int    useall         Use all the input differences (by default     */
/*                         we exclude very small overlap areas)          */
/*   int    niteration     Number of iterations to run                   */
/*   int    debug          Debugging output level                        */
/*                                                                       */
/*************************************************************************/

struct mBgModelReturn *mBgModel(char *imgfile, char *fitfile, char *corrtbl, int noslope, int useall, int niter, int debug)
</pre>
<p><b>Return Structure</b></p>
<pre>
struct mBgModelReturn
{
   int    status;        // Return status (0: OK, 1:ERROR)
   char   msg [1024];    // Return message (for error return)
   char   json[4096];    // Return parameters as JSON string
};
</pre>